In [1]:
import pandas as pd
import datetime
from collections import defaultdict

# Step 1: Load the CSV files for startups and mentors
startups = pd.read_csv("Startups.csv")
mentors = pd.read_csv("Mentor.csv")

# Step 2: Fill empty fields with "any"
startups.fillna("any", inplace=True)
mentors.fillna("any", inplace=True)

# Step 3: Assign weights to preferences
preference_weights = {1: 0.5, 2: 0.3, 3: 0.2}

# Step 4: Initialize time slots and schedules
start_time = datetime.datetime.strptime("11:00", "%H:%M")
end_time = datetime.datetime.strptime("14:00", "%H:%M")
time_slot_duration = datetime.timedelta(minutes=15)
gap_duration = datetime.timedelta(minutes=5)

# Mentor schedule dictionary
mentor_schedule = defaultdict(list)
allocated_startups = set()
mentor_startup_tracker = defaultdict(set)  # Tracks which startups each mentor has already met

# Step 5: Sort startups based on their stage and preferences
def assign_startups():
    global allocated_startups

    while True:
        unallocated_startups = set(startups.index) - allocated_startups

        # Break if no unallocated startups are left
        if not unallocated_startups:
            break

        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            sector_preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            # Create a sorted list of startups for the mentor based on preference and stage
            startup_scores = []

            for i in unallocated_startups:
                startup = startups.loc[i]
                score = 0
                if startup['sector'] in sector_preferences:
                    score += preference_weights[sector_preferences.index(startup['sector']) + 1]

                startup_scores.append((score, i))

            # Sort by score in descending order
            startup_scores.sort(reverse=True, key=lambda x: x[0])

            # Allocate startups to mentor's schedule
            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(mentor_schedule[mentor_name][-1][0], "%H:%M") + time_slot_duration + gap_duration
            current_time = max(current_time, start_time)  # Ensure time advances correctly

            for _, startup_idx in startup_scores:
                if startup_idx in allocated_startups:
                    continue

                if current_time >= end_time:
                    return

                # Assign time slot
                mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
                mentor_startup_tracker[mentor_name].add(startup_idx)
                allocated_startups.add(startup_idx)
                break

    # Allow repetition after all startups are assigned at least once
    while True:
        for _, mentor in mentors.iterrows():
            mentor_name = mentor['Name']
            sector_preferences = [mentor['Sector 1'], mentor['Sector 2'], mentor['Sector 3']]

            current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(mentor_schedule[mentor_name][-1][0], "%H:%M") + time_slot_duration + gap_duration

            if current_time >= end_time:
                return

            # Create a sorted list of startups for the mentor based on preference and stage
            startup_scores = []

            for i in startups.index:
                if i in mentor_startup_tracker[mentor_name]:
                    continue  # Skip startups already met by this mentor

                startup = startups.loc[i]
                score = 0
                if startup['sector'] in sector_preferences:
                    score += preference_weights[sector_preferences.index(startup['sector']) + 1]

                startup_scores.append((score, i))

            # Sort by score in descending order
            startup_scores.sort(reverse=True, key=lambda x: x[0])

            for _, startup_idx in startup_scores:
                current_time = start_time if not mentor_schedule[mentor_name] else datetime.datetime.strptime(mentor_schedule[mentor_name][-1][0], "%H:%M") + time_slot_duration + gap_duration

                if current_time >= end_time:
                    return

                # Assign time slot
                mentor_schedule[mentor_name].append((current_time.strftime("%H:%M"), startup_idx))
                mentor_startup_tracker[mentor_name].add(startup_idx)
                break

# Step 6: Assign startups to mentors
assign_startups()

# Step 7: Display the output
for mentor, schedule in mentor_schedule.items():
    print(f"Mentor: {mentor}")
    for time, startup_idx in schedule:
        print(f"  Time: {time}, Startup Index: {startup_idx}")
    print("\n")


Mentor: Mentor 1
  Time: 11:00, Startup Index: 9
  Time: 11:20, Startup Index: 40
  Time: 11:40, Startup Index: 124
  Time: 12:00, Startup Index: 12
  Time: 12:20, Startup Index: 25
  Time: 12:40, Startup Index: 31
  Time: 13:00, Startup Index: 43
  Time: 13:20, Startup Index: 59
  Time: 13:40, Startup Index: 69


Mentor: Mentor 2
  Time: 11:00, Startup Index: 5
  Time: 11:20, Startup Index: 73
  Time: 11:40, Startup Index: 128
  Time: 12:00, Startup Index: 13
  Time: 12:20, Startup Index: 39
  Time: 12:40, Startup Index: 81
  Time: 13:00, Startup Index: 89
  Time: 13:20, Startup Index: 132
  Time: 13:40, Startup Index: 145


Mentor: Mentor 3
  Time: 11:00, Startup Index: 10
  Time: 11:20, Startup Index: 115
  Time: 11:40, Startup Index: 149
  Time: 12:00, Startup Index: 27
  Time: 12:20, Startup Index: 53
  Time: 12:40, Startup Index: 91
  Time: 13:00, Startup Index: 92
  Time: 13:20, Startup Index: 97
  Time: 13:40, Startup Index: 106


Mentor: Mentor 4
  Time: 11:00, Startup Index: 